In [1]:
# 라이브러리 로드
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split # 학습, 테스트set 구분
from sklearn.tree import export_graphviz # tree 시각화를 위해
# export_graphviz : 의사결정나무에 대한 graphviz dot data 생성하는 함수
import graphviz # tree 시각화
import sklearn.metrics as mt # 성능지표를 계산하기 위해 import
from sklearn.model_selection import cross_val_score, cross_validate # 교차검증
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
os.environ["PATH"]+=os.pathsep+'C:/Program Files (x86)/Graphviz/bin/'

In [4]:
df=pd.read_csv("pre_labeled_weather.csv", index_col = 0)
df.head()

,location,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindGustDir,Sunshine,Evaporation,TodayRain,TomorrowRain,...,Temp9am,Pressure3pm,WindDir3pm,Cloud3pm,Humidity3pm,WindSpeed3pm,Temp3pm,Year,Month,Day
0,45,12.6,23.2,0.5,6.4,7,13.17,3.2,0,0,...,18.3,998.7,9,6.0,51.0,4.2,19.3,2012,5,5
1,45,10.6,24.1,1.8,6.5,15,13.17,3.2,0,0,...,18.6,1000.8,0,2.0,35.0,2.7,20.5,2012,5,6
2,45,14.2,25.1,1.8,7.0,7,13.17,3.2,0,0,...,22.2,1002.8,0,0.0,31.0,2.1,22.9,2012,5,7
3,45,12.1,21.1,1.8,6.1,7,13.17,3.2,0,0,...,17.0,1005.5,2,3.0,70.0,2.8,18.3,2012,5,8
4,45,11.9,15.4,1.8,6.5,7,13.17,3.2,0,1,...,14.7,1013.2,6,9.0,89.0,4.6,13.9,2012,5,9


In [3]:
df=pd.read_csv("./data/MiceImputed_weather.csv", index_col = 0)
df.head()

,location,Date,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindGustDir,Sunshine,Evaporation,TodayRain,...,Cloud9am,Humidity9am,WindSpeed9am,Temp9am,Pressure3pm,WindDir3pm,Cloud3pm,Humidity3pm,WindSpeed3pm,Temp3pm
0,45.0,0.0,12.6,23.2,0.5,6.4,7.0,17.627755,4.696022,0.0,...,0.0,66.0,1.6,18.3,998.7,9.0,6.0,51.0,4.2,19.3
1,45.0,1.0,10.6,24.1,0.0,6.5,15.0,21.060788,6.238979,0.0,...,3.0,30.0,5.1,18.6,1000.8,0.0,2.0,35.0,2.7,20.5
2,45.0,2.0,14.2,25.1,0.0,7.0,7.0,24.272884,6.987526,0.0,...,0.0,26.0,3.1,22.2,1002.8,0.0,0.0,31.0,2.1,22.9
3,45.0,3.0,12.1,21.1,0.0,6.1,7.0,14.398951,3.787547,0.0,...,9.0,66.0,0.8,17.0,1005.5,2.0,3.0,70.0,2.8,18.3
4,45.0,4.0,11.9,15.4,0.0,6.5,7.0,7.348844,1.284132,0.0,...,8.0,90.0,3.8,14.7,1013.2,6.0,9.0,89.0,4.6,13.9


In [11]:
df=df.drop("Date",axis=1)

In [12]:
X = df.drop(['TomorrowRain'], axis=1)
y = df['TomorrowRain']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,stratify=y, random_state = 0)

In [14]:
dt_clf = DecisionTreeClassifier(random_state=156)
dt_clf.fit(X_train, y_train)

DecisionTreeClassifier(random_state=156)

In [15]:
export_graphviz(dt_clf, out_file="tree.dot", 
                class_names= ["Positive","Negative"], 
                feature_names = X.keys(), 
                impurity = True,
                filled= True)

In [16]:
y_pred = dt_clf.predict(X_test) 

In [17]:
import sklearn.metrics as mt
def get_clf_eval(y_test, pred=None):
    confusion = mt.confusion_matrix( y_test, pred)
    accuracy = mt.accuracy_score(y_test , pred)
    precision = mt.precision_score(y_test , pred)
    recall = mt.recall_score(y_test , pred)
    f1 = mt.f1_score(y_test,pred)

    print('오차 행렬:')
    print(confusion)
    
    print('\n정확도: {0:.4f} \n정밀도: {1:.4f} \n재현율: {2:.4f} \n    F1: {3:.4f}'.format(accuracy, precision, recall, f1))


In [18]:
get_clf_eval(y_test, y_pred)

오차 행렬:
[[46164  8538]
 [ 7867  6173]]

정확도: 0.7614 
정밀도: 0.4196 
재현율: 0.4397 
    F1: 0.4294
